# Hybrid Recommendation using LightFM

A **hybrid recommendation system** is a specialized type of recommendation system that combines both content-based and collaborative filtering methods. This hybrid approach aims to overcome the limitations of using these methods separately and can prove to be more effective in certain cases.

**Hybrid recommender systems** can be implemented in various ways:

1. Combining Predictions: In this approach, collaborative and content-based methods generate predictions separately. The final recommendations are then created by combining these predictions.

2. Adding Capabilities: Alternatively, you can enhance a content-based approach with collaborative filtering capabilities or vice versa.

Numerous studies have compared the performance of conventional recommendation approaches with hybrid methods. These studies often conclude that hybrid methods are capable of generating more accurate recommendations.

---

In [ ]:
!pip install lightfm==1.16
!pip install numpy==1.19.5
!pip install pandas==1.3.5
!pip install scikit_learn==0.24.1
!pip install scipy==1.6.2

---

In [ ]:
# Importing libraries
import pandas as pd 
import numpy as np 
from scipy.sparse import coo_matrix # for constructing sparse matrix

# Importing lightfm libraries and modules
from lightfm import LightFM # model
from lightfm.evaluation import auc_score

import time

## Importing the Data from Excel

In [ ]:
# Read data from Excel file 'Rec_sys_data.xlsx' into DataFrames

order = pd.read_excel('Rec_sys_data.xlsx', 'order')  # Read 'order' sheet into the 'order' DataFrame
customer = pd.read_excel('Rec_sys_data.xlsx', 'customer')  # Read 'customer' sheet into the 'customer' DataFrame
product = pd.read_excel('Rec_sys_data.xlsx', 'product')  # Read 'product' sheet into the 'product' DataFrame


## Merging the datasets

In [ ]:
# Merge the 'order' and 'customer' DataFrames based on the 'CustomerID' column, using a left join
full_table = pd.merge(order, customer, left_on=['CustomerID'], right_on=['CustomerID'], how='left')

# Merge the resulting DataFrame with the 'product' DataFrame based on the 'StockCode' column, using a left join
full_table = pd.merge(full_table, product, left_on=['StockCode'], right_on=['StockCode'], how='left')


In [ ]:
# Check for first 5 rows for `order` data
order.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [ ]:
# Check for first 5 rows for `customer` data
customer.head()

,CustomerID,Gender,Age,Income,Zipcode,Customer Segment
0,13089,male,53,High,8625,Small Business
1,15810,female,22,Low,87797,Small Business
2,15556,female,29,High,29257,Corporate
3,13137,male,29,Medium,97818,Middle class
4,16241,male,36,Low,79200,Small Business


In [ ]:
# Check for first 5 rows for `produc` data
product.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [ ]:
# Function to create a sorted list of unique users from a specified column in the data
def unique_users(data, column):
    return np.sort(data[column].unique()) 

# Function to create a list of unique products/items from a specified column in the data
def unique_items(data, column):
    item_list = data[column].unique() 
    return item_list

# Function to create a list of features by concatenating specified columns from the customer data
def features_to_add(customer, column1, column2, column3):
    customer1 = customer[column1]
    customer2 = customer[column2]
    customer3 = customer[column3]
    combined_features = pd.concat([customer1, customer3, customer2], ignore_index=True).unique()
    return combined_features 

# Function to create ID mappings for users, items, and features
def mapping(users, items, features):
    user_to_index_mapping = {} 
    index_to_user_mapping = {}  
    for user_index, user_id in enumerate(users):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {} 
    index_to_item_mapping = {}  
    for item_index, item_id in enumerate(items):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    feature_to_index_mapping = {}  
    index_to_feature_mapping = {} 
    for feature_index, feature_id in enumerate(features):
        feature_to_index_mapping[feature_id] = feature_index
        index_to_feature_mapping[feature_index] = feature_id
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping


In [ ]:
# Create a list of unique users from the 'CustomerID' column in the 'order' DataFrame
users = unique_users(order, "CustomerID")

# Create a list of unique items (Product Names) from the 'Product Name' column in the 'product' DataFrame
items = unique_items(product, "Product Name")

# Create a list of unique features by concatenating 'Customer Segment', 'Age', and 'Gender' columns from the 'customer' DataFrame
features = features_to_add(customer, 'Customer Segment', 'Age', 'Gender')


In [ ]:
users

array([12346, 12347, 12348, ..., 18282, 18283, 18287])

In [ ]:
items

array(['Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover',
       'Eye Buy Express Prescription Glasses Mens Womens Burgundy Crystal Clear Yellow Rounded Rectangular Reading Glasses Anti Glare grade',
       "MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options",
       'Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches',
       'Stupell Industries Chevron Initial Wall D cor',
       "MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options",
       "MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options",
       'MightySkins Skin For OtterBox Commuter iPhone 8 Plus | Protective, Durable, and Unique Vinyl Decal wrap cover | Easy To Apply, Remove, and Change Styles | Made in the USA',
       "Dr. Comfort Paradise Women's Casual Shoe: 4.5 X-Wide (E-2E) Black Velcro",
       "Mig

In [ ]:
features

array(['Small Business', 'Corporate', 'Middle class', 'male', 'female',
       53, 22, 29, 36, 48, 45, 47, 23, 39, 34, 52, 51, 35, 19, 26, 37, 18,
       20, 21, 41, 31, 28, 50, 38, 30, 25, 32, 55, 43, 54, 49, 40, 33, 44,
       46, 42, 27, 24], dtype=object)

In [ ]:
# Generate mappings for users, items, and features to convert IDs to integer indices
user_to_index_mapping, index_to_user_mapping, \
item_to_index_mapping, index_to_item_mapping, \
feature_to_index_mapping, index_to_feature_mapping = mapping(users, items, features)


In [ ]:
full_table.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID,Gender,Age,Income,Zipcode,Customer Segment,Product Name,Description,Category,Brand,Unit Price
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,"3 1/2""W x 20""D x 20""H Funston Craftsman Smooth...",Our Rustic Collection is an instant classic. O...,Home Improvement|Hardware|Brackets and Angle I...,Ekena Milwork,199.11
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,Awkward Styles Shamrock Flag St. Patrick's Day...,Our St Patrick's Day Collection is perfect for...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens Gr...,Awkward Styles,23.95
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,Ebe Men Black Rectangle Half Rim Spring Hinge ...,Count on EBE for all of your eye correction ne...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,26.99
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,MightySkins Skin Decal Wrap Compatible with Ap...,Mightyskins are removable vinyl skins for prot...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,14.99
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850,female,48,Medium,84306,Middle class,awesome since 1948 - 69th birthday gift t-shir...,awesome since 1948 - 69th birthday gift t-shir...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Shirtinvaders,49.33


In [ ]:
# Selecting the columns 'CustomerID', 'Product Name', and 'Quantity' from the 'full_table' DataFrame
user_to_product_rating_train = full_table[['CustomerID', 'Product Name', 'Quantity']]


In [ ]:
# Selecting the columns 'Product Name', 'Customer Segment', and 'Quantity' from the 'full_table' DataFrame
product_to_feature = full_table[['Product Name', 'Customer Segment', 'Quantity']]


In [ ]:
# Grouping the 'user_to_product_rating_train' DataFrame by 'CustomerID' and 'Product Name' and aggregating 'Quantity' by sum
user_to_product_rating_train = user_to_product_rating_train.groupby(['CustomerID', 'Product Name']).agg({'Quantity': 'sum'}).reset_index()


In [ ]:
user_to_product_rating_train.tail()

,CustomerID,Product Name,Quantity
138397,18287,Sport-Tek Ladies PosiCharge Competitor Tee,24
138398,18287,Ultra Sleek And Spacious Pearl White Lacquer 1...,6
138399,18287,"Union 3"" Female Ports Stainless Steel Pipe Fit...",12
138400,18287,awesome since 1948 - 69th birthday gift t-shir...,4
138401,18287,billyboards Porcelain Menu Chalkboard,6


---

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Perform a train-test split on the 'user_to_product_rating_train' DataFrame
# - The test set size is 33% (0.33) of the data
user_to_product_rating_train, user_to_product_rating_test = train_test_split(user_to_product_rating_train, test_size=0.33, random_state=42)


In [ ]:
# Check the shape of train data
user_to_product_rating_train.shape

(92729, 3)

In [ ]:
# Check the shape of test data
user_to_product_rating_test.shape

(45673, 3)

In [ ]:
# Grouping the 'product_to_feature' DataFrame by 'Product Name' and 'Customer Segment' and aggregating 'Quantity' by sum
product_to_feature = product_to_feature.groupby(['Product Name', 'Customer Segment']).agg({'Quantity': 'sum'}).reset_index()


In [ ]:
product_to_feature.head()

,Product Name,Customer Segment,Quantity
0,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Corporate,712
1,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Middle class,272
2,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Small Business,388
3,"""Soccer"" Vinyl Graphic - Large - Ivory",Corporate,1940
4,"""Soccer"" Vinyl Graphic - Large - Ivory",Middle class,1418


In [ ]:
from scipy.sparse import coo_matrix  # Import coo_matrix from scipy.sparse for constructing sparse matrices

# Function to create a sparse interaction matrix from data
def interactions(data, row, col, value, row_map, col_map):
    # Map values from data to row and column indices using the provided mappings
    row_indices = data[row].apply(lambda x: row_map[x]).values
    col_indices = data[col].apply(lambda x: col_map[x]).values
    values = data[value].values
    
    # Create a coo_matrix (sparse matrix) using the mapped values
    return coo_matrix((values, (row_indices, col_indices)), shape=(len(row_map), len(col_map)))


In [ ]:
# Generate the user-item interaction matrix for the train data
user_to_product_interaction_train = interactions(user_to_product_rating_train, "CustomerID", 
                                                "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)

# Generate the item-to-feature interaction matrix
product_to_feature_interaction = interactions(product_to_feature, "Product Name", "Customer Segment", "Quantity", 
                                            item_to_index_mapping, feature_to_index_mapping)


In [ ]:
# Generate the user-item interaction matrix for the test data
user_to_product_interaction_test = interactions(user_to_product_rating_test, "CustomerID", 
                                                "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)


In [ ]:
user_to_product_interaction_train

<3647x797 sparse matrix of type '<class 'numpy.int64'>'
	with 92729 stored elements in COOrdinate format>

In [ ]:
user_to_product_interaction_test

<3647x797 sparse matrix of type '<class 'numpy.int64'>'
	with 45673 stored elements in COOrdinate format>

In [ ]:
product_to_feature_interaction

<797x43 sparse matrix of type '<class 'numpy.int64'>'
	with 2083 stored elements in COOrdinate format>

---

# Model building

Parameters:
- Loss=warp
- epochs=1 
- num_threads=4

In [ ]:
# Initialize the model with the "warp" loss function
model_with_features = LightFM(loss="warp")

# Fit the model with hybrid collaborative filtering + content-based (product + features)
start = time.time()  # Record the start time

# Fit the model using the fit_partial method, combining user-item interactions and item features
model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None, 
                                item_features=product_to_feature_interaction, 
                                sample_weight=None, 
                                epochs=1, 
                                num_threads=4,
                                verbose=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for model fitting


time taken = 0.12 seconds


In [ ]:
start = time.time()  # Record the start time

# Calculate AUC score for the model with hybrid collaborative filtering + content-based features
auc_with_features = auc_score(model=model_with_features, 
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train, 
                              item_features=product_to_feature_interaction,
                              num_threads=4, check_intersections=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for AUC calculation

# Print the average AUC score without adding item-feature interaction
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.21 seconds
average AUC without adding item-feature interaction = 0.15


---

# Model building
Parameters:
- Loss=logistic
- epochs=1 
- num_threads=4

In [ ]:
# Initialize the model with the "logistic" loss function
model_with_features = LightFM(loss="logistic")

# Fit the model with hybrid collaborative filtering + content-based (product + features)
start = time.time()  # Record the start time

# Fit the model using the fit_partial method, combining user-item interactions and item features
model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None, 
                                item_features=product_to_feature_interaction, 
                                sample_weight=None, 
                                epochs=1, 
                                num_threads=4,
                                verbose=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for model fitting


time taken = 0.07 seconds


In [ ]:
start = time.time()  # Record the start time

# Calculate AUC score for the model with hybrid collaborative filtering + content-based features
auc_with_features = auc_score(model=model_with_features, 
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train, 
                              item_features=product_to_feature_interaction,
                              num_threads=4, check_intersections=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for AUC calculation

# Print the average AUC score without adding item-feature interaction
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.26 seconds
average AUC without adding item-feature interaction = 0.89


---

# Model building
Parameters:
- Loss=bpr
- epochs=1 
- num_threads=4

In [ ]:
# Initialize the model with the "bpr" loss function
model_with_features = LightFM(loss="bpr")

# Fit the model with hybrid collaborative filtering + content-based (product + features)
start = time.time()  # Record the start time

# Fit the model using the fit_partial method, combining user-item interactions and item features
model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None, 
                                item_features=product_to_feature_interaction, 
                                sample_weight=None, 
                                epochs=1, 
                                num_threads=4,
                                verbose=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for model fitting


time taken = 0.13 seconds


In [ ]:
start = time.time()  # Record the start time

# Calculate AUC score for the model with hybrid collaborative filtering + content-based features
auc_with_features = auc_score(model=model_with_features, 
                              test_interactions=user_to_product_interaction_test,
                              train_interactions=user_to_product_interaction_train, 
                              item_features=product_to_feature_interaction,
                              num_threads=4, check_intersections=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for AUC calculation

# Print the average AUC score without adding item-feature interaction
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.21 seconds
average AUC without adding item-feature interaction = 0.18


---

# Model building
Parameters:
- Loss=logistic
- epochs=10 
- num_threads=20

In [ ]:
# Initialize the model with the "logistic" loss function
model_with_features = LightFM(loss="logistic")

# Fit the model with hybrid collaborative filtering + content-based (product + features)
start = time.time()  # Record the start time

# Fit the model using the fit_partial method, combining user-item interactions and item features
model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None, 
                                item_features=product_to_feature_interaction, 
                                sample_weight=None, 
                                epochs=10, 
                                num_threads=20,
                                verbose=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for model fitting


time taken = 0.67 seconds


In [ ]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.22 seconds
average AUC without adding item-feature interaction = 0.89


In [ ]:
def train_test_merge(training_data, testing_data):
    # Initialize a dictionary to hold the merged data
    train_dict = {}

    # Iterate through the training data and populate the dictionary
    for row, col, data in zip(training_data.row, training_data.col, training_data.data):
        train_dict[(row, col)] = data

    # Replace data in the dictionary with values from the test set, taking the maximum
    for row, col, data in zip(testing_data.row, testing_data.col, testing_data.data):
        train_dict[(row, col)] = max(data, train_dict.get((row, col), 0))

    # Convert the merged data into separate lists for rows, columns, and data
    row_list = []
    col_list = []
    data_list = []

    for row, col in train_dict:
        row_list.append(row)
        col_list.append(col)
        data_list.append(train_dict[(row, col)])

    # Convert the lists to NumPy arrays
    row_list = np.array(row_list)
    col_list = np.array(col_list)
    data_list = np.array(data_list)

    # Create a coo_matrix using the merged data
    return coo_matrix((data_list, (row_list, col_list)), shape=(training_data.shape[0], training_data.shape[1]))


In [ ]:
user_to_product_interaction = train_test_merge(user_to_product_interaction_train, 
                                                 user_to_product_interaction_test)

In [ ]:
user_to_product_interaction

<3647x797 sparse matrix of type '<class 'numpy.float64'>'
	with 138402 stored elements in COOrdinate format>

---

# Final Model after combining train and test Data
Parameters:
- Loss=warp
- epochs=10 
- num_threads=20

In [ ]:
# Initialize the final model with the "logistic" loss function and 30 components
final_model = LightFM(loss="logistic", no_components=30)

# Fit the final model using the combined dataset
start = time.time()  # Record the start time

# Fit the final model using the fit method, combining user-item interactions and item features
final_model.fit(user_to_product_interaction,
                user_features=None, 
                item_features=product_to_feature_interaction, 
                sample_weight=None, 
                epochs=10, 
                num_threads=20,
                verbose=False)

end = time.time()  # Record the end time
print("time taken = {0:.{1}f} seconds".format(end - start, 2))  # Print the time taken for model fitting


time taken = 2.16 seconds


In [ ]:
def get_recommendations(model, user, items, user_to_product_interaction_matrix, user2index_map, product_to_feature_interaction_matrix):
    # Get the user's index using the user2index_map
    user_index = user2index_map.get(user, None)
    
    # If the user is not found, return None
    if user_index is None:
        return None
    
    # Get the user's index
    users = user_index
    
    # Get products that the user has already bought
    known_positives = items[user_to_product_interaction_matrix.tocsr()[user_index].indices]
    print('User index =', users)
    
    # Predict scores for all items using the model
    scores = model.predict(user_ids=users, item_ids=np.arange(user_to_product_interaction_matrix.shape[1]), item_features=product_to_feature_interaction_matrix)
    
    # Sort the items by scores in descending order to get top recommendations
    top_items = items[np.argsort(-scores)]
    
    # Print the recommendations
    print("User %s" % user)
    print("     Known positives:")  # Already known products
    
    for x in known_positives[:10]:
        print("                  %s" % x)
    
    print("     Recommended:")  # Products recommended to the user
    
    for x in top_items[:10]:
        print("                  %s" % x)


In [ ]:
# Check for the reccomendation
get_recommendations(final_model,17017,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 2888
User 17017
     Known positives:
                  Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover
                  MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options
                  Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches
                  MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  MightySkins Skin Decal Wrap Compatible with Lenovo Sticker Protective Cover 100's of Color Options
                  Ebe Reading Glasses Mens Womens Tortoise Bold Rectangular Full Frame Anti Glare grade ckbdp9088
                  Window Tint Film Chevy (back doors) DIY
                  Union 3" Female Ports Stainless Steel Pipe Fitting
                  Ebe Wo

In [ ]:
get_recommendations(final_model,18287,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 3646
User 18287
     Known positives:
                  MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options
                  Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  Union 3" Female Ports Stainless Steel Pipe Fitting
                  Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209
                  MightySkins Skin Decal Wrap Compatible with Apple Sticker Protective Cover 100's of Color Options
                  New Way 1120 - Crewneck Your Job Could Be Worse Toilet Paper Sweatshirt 4XL Light Pink
                  6pc Boy Formal Necktie Black & White Suit Set Satin Bow tie Baby Sm-20 Teen
                  Buckle-Down PC-W30348-NM Cameroon Flags Plastic Clip Collar
                  Business Essentials 8" 

In [ ]:
get_recommendations(final_model,13933,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 1000
User 13933
     Known positives:
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209
                  MightySkins Skin Decal Wrap Compatible with Smok Sticker Protective Cover 100's of Color Options
                  3 1/2"W x 20"D x 20"H Funston Craftsman Smooth Bracket, Douglas Fir
                  Ebe Reading Glasses Mens Womens Blue Silver Temples Anti Glare remarkable grade ckb8147
                  Port Authority J335 Hooded Core Soft Shell Jacket, Dress Blue Navy/ Battleship Grey, XS
                  Buckle-Down PC-W30348-NM Cameroon Flags Plastic Clip Collar
                  Business Essentials 8" x 8" x 5" Corrugated Mailers, 12-Pack
                  6 1/4 x 6 1/4 Gatefold Invitation - Mandarin Orange (500 Qty.)
                  Awkward Styles Sugar Skull Shirts for Men Jolly Roger Skull and Crossbo

---